In [32]:
import requests
import base64
import json
from openai import OpenAI

# Document Parser API로 PDF 텍스트 추출
def parse_document(file_path):
    api_key = "up_IANAGmcf5Z0MK40wgyiNBYJAlFMTe"
    url = "https://api.upstage.ai/v1/document-ai/document-parse"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(file_path, "rb")}
    data = {
        "ocr": "force",
        "coordinates": True,
        "chart_recognition": True,
        "output_formats": "['text']",
        "base64_encoding": "['table']",
        "model": "document-parse"
    }
    response = requests.post(url, headers=headers, files=files, data=data)
    if response.status_code == 200:
        return response.json()['content']['text']
    return None

# Information Extract API로 PNG에서 정보 추출
def extract_info_from_image(img_path):
    client = OpenAI(
        api_key="up_IANAGmcf5Z0MK40wgyiNBYJAlFMTe",
        base_url="https://api.upstage.ai/v1/information-extraction"
    )
    with open(img_path, 'rb') as img_file:
        base64_data = base64.b64encode(img_file.read()).decode('utf-8')

    response = client.chat.completions.create(
        model="information-extract",
        messages=[{
            "role": "user",
            "content": [{
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_data}"}
            }]
        }],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "bitcoin_data",
                "schema": {
                    "type": "object",
                    "properties": {
                        "price": {"type": "string", "description": "비트코인 가격"},
                        "date": {"type": "string", "description": "날짜"}
                    },
                    "required": ["price", "date"]
                }
            }
        }
    )
    return json.loads(response.choices[0].message.content)

# 테스트
if __name__ == "__main__":
    pdf_text = parse_document("document.pdf")
    img_data = extract_info_from_image("document1.png")
    print("PDF Text:", pdf_text)
    print("Image Data:", img_data)

PDF Text: 오스코텍
(039200)
이희영
heeyoung.lee@daishin.com
투자의견 BUY 매수, 유지
 6개월 목표주가 59,000 유지
 현재주가 (25.03.26) 31,650
제약업종
KOSDAQ 716.48
 시가총액 1,211십억원
 시가총액비중 0.32%
 자본금(보통주) 19십억원
 52주 최고/최저 44,100원 /21,700원
 120일 평균거래대금 115억원
 외국인지분율 10.51%
 주요주주 김정근 외5 인 12.85% 지케이에셋 외3 인 9.79%
주가수익률(%) 1M 3M 6M 12M
 절대수익률 19.0 33.8 -7.7 11.8
 상대수익률 28.1 26.2 0.3 43.0
Chart Type: bar
  24.03 24.06 24.09 24.12 25.03
 오스코택 () 26.5% 39.6% 51.5% 34% 32%
 Relative to KOSDAQ(우) 25.0% 33.8% 39.5% 26.5% 27.0%
Issue & News
경영진 교체에 따른
영향은 제한적
■ 투자의견 매수, 목표주가 59,000원 유지
■ 25년 오스코텍 주주총회, 김정근 대표 재선임안 부결
■ 회사 경영에 미치는 영향은 제한적, 자회사 상장 방향성이 중요
투자의견 매수, 목표주가 59,000원 유지
김정근 대표이사의 재선임안이 부결되었으나, 윤태영 각자대표가 경영권을
이어받으며, 경영 공백은 제한적일 것으로 판단. 핵심 파이프라인 라즈클루즈
에 대해서는 긍정적인 임상 결과 발표가 이어지고 있으며, 글로벌 상업화 역
시 계획대로 진행중, 더불어, 후속 파이프라인 임상도 순항중이며 동사의 중
장기 성장 모멘텀은 여전히 유효. 제노스코 상장 관련 우려는 이미 상당 부분
주가에 반영되었으며, 이번 사태를 계기로 주주 친화적인 결정이 강화된다면,
주가 리레이팅의 계기가 될 수 있음. 따라서 투자의견 매수, 목표주가 59,000
원 유지함
25년 오스코텍 주주총회, 김정근 대표 재선임안 부결
25년 3월 27일 열린 오스코텍 27기 정기 

In [33]:
import re
from textblob import TextBlob

# PDF 텍스트에서 가격 추출
def extract_price_data(text):
    prices = re.findall(r'\$\d+(?:,\d+)?', text)
    return [float(price.replace('$', '').replace(',', '')) for price in prices]

# PNG 이미지 데이터 분석
def analyze_image_data(img_data):
    price = img_data.get("price", "0").replace('$', '').replace(',', '')
    return float(price) if price else 0

# 감정 분석 (PDF 텍스트 기반)
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # -1(부정) ~ 1(긍정)

# 매매 신호 생성
def generate_signal(prices, img_price, sentiment_score):
    avg_price = sum(prices) / len(prices) if prices else img_price
    if sentiment_score > 0.5 and img_price > avg_price:  # 긍정적 + 가격 상승
        return "매수"
    elif sentiment_score < 0 and img_price < avg_price:  # 부정적 + 가격 하락
        return "매도"
    return "관망"

# 테스트
if __name__ == "__main__":
    pdf_text = "Bitcoin price is $60,000 and rising."
    img_data = {"price": "$62,000", "date": "2025-03-30"}
    prices = extract_price_data(pdf_text)
    img_price = analyze_image_data(img_data)
    sentiment = analyze_sentiment(pdf_text)
    signal = generate_signal(prices, img_price, sentiment)
    print("Signal:", signal)

Signal: 관망


In [34]:
class TradingDecision:
    def __init__(self, balance=1000, btc_held=0):
        self.balance = balance
        self.btc_held = btc_held

    def make_decision(self, signal, current_price):
        if signal == "매수" and self.balance >= current_price * 0.1:
            amount = 0.1
            cost = amount * current_price
            self.balance -= cost
            self.btc_held += amount
            return f"0.1 BTC 매수 at ${current_price}"
        elif signal == "매도" and self.btc_held >= 0.1:
            amount = 0.1
            revenue = amount * current_price
            self.balance += revenue
            self.btc_held -= amount
            return f"0.1 BTC 매도 at ${current_price}"
        return "관망"

# 테스트
if __name__ == "__main__":
    trader = TradingDecision(balance=1000, btc_held=0.5)
    decision = trader.make_decision("매수", 62000)
    print("Decision:", decision)

Decision: 관망


In [35]:
class TradingExecutor:
    def __init__(self):
        self.simulated = True

    def execute_trade(self, decision):
        if self.simulated:
            return f"시뮬레이션: {decision} 실행 완료"
        return "실제 거래 미구현"

# 테스트
if __name__ == "__main__":
    executor = TradingExecutor()
    result = executor.execute_trade("0.1 BTC 매수 at $62000")
    print("Result:", result)

Result: 시뮬레이션: 0.1 BTC 매수 at $62000 실행 완료


In [25]:
class FeedbackLoop:
    def __init__(self):
        self.trade_history = []

    def log_trade(self, decision, current_price, new_price):
        profit = 0
        if "매수" in decision:
            profit = (new_price - current_price) * 0.1
        elif "매도" in decision:
            profit = (current_price - new_price) * 0.1
        self.trade_history.append({"decision": decision, "profit": profit})

    def evaluate(self):
        total_profit = sum(trade["profit"] for trade in self.trade_history)
        return f"총 수익: ${total_profit:.2f}"

# 테스트
if __name__ == "__main__":
    feedback = FeedbackLoop()
    feedback.log_trade("0.1 BTC 매수 at $62000", 62000, 63000)
    print(feedback.evaluate())

총 수익: $100.00


In [36]:
# 데이터 수집
pdf_text = parse_document("document.pdf")
img_data = extract_info_from_image("document1.png")

# 데이터 분석
prices = extract_price_data(pdf_text)
img_price = analyze_image_data(img_data)
sentiment = analyze_sentiment(pdf_text)
signal = generate_signal(prices, img_price, sentiment)

# 의사결정
trader = TradingDecision(balance=1000, btc_held=0)
decision = trader.make_decision(signal, img_price)

# 실행
executor = TradingExecutor()
result = executor.execute_trade(decision)

# 피드백
feedback = FeedbackLoop()
feedback.log_trade(decision, img_price, img_price + 1000)  # 가격 변화 가정
print("Execution Result:", result)
print(feedback.evaluate())

Execution Result: 시뮬레이션: 관망 실행 완료
총 수익: $0.00


In [39]:
from datetime import datetime
# 데이터 수집 및 분석 부분 수정
def analyze_document_data(pdf_text):
    """PDF 문서에서 실제 데이터 분석"""
    # 주가 정보 추출
    price_pattern = r'현재주가\s*\([\d\.]+\)\s*([\d,]+)'
    current_price = 0
    price_match = re.search(price_pattern, pdf_text)
    if price_match:
        current_price = float(price_match.group(1).replace(',', ''))
    
    # 투자의견 추출
    opinion_pattern = r'투자의견\s*([\w\s]+)'
    investment_opinion = ''
    opinion_match = re.search(opinion_pattern, pdf_text)
    if opinion_match:
        investment_opinion = opinion_match.group(1).strip()
    
    # 목표가 추출
    target_pattern = r'목표주가\s*([\d,]+)'
    target_price = 0
    target_match = re.search(target_pattern, pdf_text)
    if target_match:
        target_price = float(target_match.group(1).replace(',', ''))
    
    return {
        'current_price': current_price,
        'target_price': target_price,
        'investment_opinion': investment_opinion
    }

def analyze_sentiment(text):
    """문서의 감성 분석"""
    # 긍정적 키워드
    positive_keywords = ['상승', '성장', '개선', '호조', '긍정', '매수', '강세']
    # 부정적 키워드
    negative_keywords = ['하락', '감소', '부진', '악화', '부정', '매도', '약세']
    
    score = 0
    for word in positive_keywords:
        score += text.count(word) * 0.1
    for word in negative_keywords:
        score -= text.count(word) * 0.1
    
    return max(min(score, 1.0), -1.0)  # -1 ~ 1 사이로 정규화

def generate_signal(doc_data, sentiment_score):
    """매매 신호 생성"""
    current_price = doc_data['current_price']
    target_price = doc_data['target_price']
    opinion = doc_data['investment_opinion']
    
    # 매수 신호 조건
    if (sentiment_score > 0.3 and 
        current_price < target_price * 0.9 and 
        opinion.lower().find('매수') >= 0):
        return "매수"
    
    # 매도 신호 조건
    elif (sentiment_score < -0.3 or 
          current_price > target_price * 1.1 or 
          opinion.lower().find('매도') >= 0):
        return "매도"
    
    return "관망"

class TradingDecision:
    def __init__(self, balance=1000000, stocks_held=0):
        self.balance = balance
        self.stocks_held = stocks_held
        
    def make_decision(self, signal, current_price):
        """실제 거래 가능한 수량을 계산하여 의사결정"""
        if signal == "매수" and self.balance >= current_price:
            possible_quantity = self.balance // current_price
            if possible_quantity > 0:
                trade_quantity = min(possible_quantity, 10)  # 최대 10주로 제한
                cost = trade_quantity * current_price
                self.balance -= cost
                self.stocks_held += trade_quantity
                return f"{trade_quantity}주 매수 @ {current_price:,}원"
                
        elif signal == "매도" and self.stocks_held > 0:
            trade_quantity = min(self.stocks_held, 10)  # 최대 10주로 제한
            revenue = trade_quantity * current_price
            self.balance += revenue
            self.stocks_held -= trade_quantity
            return f"{trade_quantity}주 매도 @ {current_price:,}원"
            
        return "관망"

class FeedbackLoop:
    def __init__(self):
        self.trade_history = []
        
    def log_trade(self, decision, price):
        """실제 거래 내역 기록"""
        if decision != "관망":
            trade_info = {
                "timestamp": datetime.now(),
                "decision": decision,
                "price": price
            }
            self.trade_history.append(trade_info)
    
    def evaluate(self):
        """실제 수익 계산"""
        if not self.trade_history:
            return "거래 내역이 없습니다."
            
        total_profit = 0
        buy_amount = 0
        sell_amount = 0
        
        for trade in self.trade_history:
            price = trade["price"]
            if "매수" in trade["decision"]:
                quantity = int(trade["decision"].split("주")[0])
                buy_amount += quantity * price
            elif "매도" in trade["decision"]:
                quantity = int(trade["decision"].split("주")[0])
                sell_amount += quantity * price
        
        total_profit = sell_amount - buy_amount
        return f"총 수익: {total_profit:,}원"

# 메인 실행 코드
if __name__ == "__main__":
    # 실제 문서 파싱
    pdf_text = parse_document("document.pdf")
    
    # 문서 데이터 분석
    doc_data = analyze_document_data(pdf_text)
    sentiment = analyze_sentiment(pdf_text)
    
    # 매매 신호 생성
    signal = generate_signal(doc_data, sentiment)
    
    # 거래 의사결정
    trader = TradingDecision(balance=10000000)  # 1천만원 초기자본
    decision = trader.make_decision(signal, doc_data['current_price'])
    
    # 거래 실행
    executor = TradingExecutor()
    result = executor.execute_trade(decision)
    
    # 결과 기록 및 평가
    feedback = FeedbackLoop()
    feedback.log_trade(decision, doc_data['current_price'])
    
    # 결과 출력
    print(f"현재가: {doc_data['current_price']:,}원")
    print(f"목표가: {doc_data['target_price']:,}원")
    print(f"투자의견: {doc_data['investment_opinion']}")
    print(f"감성점수: {sentiment:.2f}")
    print(f"매매신호: {signal}")
    print(f"실행결과: {result}")
    print(feedback.evaluate())

현재가: 31,650.0원
목표가: 59,000.0원
투자의견: BUY 매수
감성점수: 1.00
매매신호: 매수
실행결과: 시뮬레이션: 10주 매수 @ 31,650.0원 실행 완료
총 수익: -316,500.0원


In [40]:
import requests
import datetime
import re
import json
import time

def get_bitcoin_price():
    """Upbit API를 통해 실제 비트코인 가격 조회"""
    try:
        url = "https://api.upbit.com/v1/ticker"
        params = {"markets": "KRW-BTC"}
        response = requests.get(url, params=params)
        data = response.json()
        if data and len(data) > 0:
            return {
                'current_price': float(data[0]['trade_price']),
                'opening_price': float(data[0]['opening_price']),
                'high_price': float(data[0]['high_price']),
                'low_price': float(data[0]['low_price']),
                'prev_closing_price': float(data[0]['prev_closing_price']),
                'change_rate': float(data[0]['change_rate']) * 100  # 변화율(%)
            }
    except Exception as e:
        print(f"비트코인 가격 조회 실패: {e}")
        return None

def analyze_market_sentiment():
    """비트코인 시장 감성 분석"""
    try:
        # 24시간 가격 변동 데이터 가져오기
        price_data = get_bitcoin_price()
        if not price_data:
            return 0
        
        sentiment_score = 0
        
        # 가격 상승률 기반 감성 점수
        price_change = price_data['change_rate']
        sentiment_score += price_change * 0.3  # 가격 변화에 30% 가중치
        
        # 거래량 기반 감성 점수 추가 가능
        # 뉴스 데이터 기반 감성 점수 추가 가능
        
        return max(min(sentiment_score, 1.0), -1.0)  # -1 ~ 1 사이로 정규화
    except Exception as e:
        print(f"시장 감성 분석 실패: {e}")
        return 0

def generate_signal(price_data, sentiment_score):
    """매매 신호 생성"""
    if not price_data:
        return "관망"
    
    current_price = price_data['current_price']
    prev_closing = price_data['prev_closing_price']
    
    # 매수 신호 조건
    if (sentiment_score > 0.3 and 
        current_price < prev_closing * 0.95):  # 5% 이상 하락했을 때
        return "매수"
    
    # 매도 신호 조건
    elif (sentiment_score < -0.3 or 
          current_price > prev_closing * 1.05):  # 5% 이상 상승했을 때
        return "매도"
    
    return "관망"

class TradingDecision:
    def __init__(self, balance=1000000, btc_held=0):
        self.balance = balance  # 원화
        self.btc_held = btc_held  # 비트코인 수량
        
    def make_decision(self, signal, current_price):
        """실제 거래 가능한 수량을 계산하여 의사결정"""
        if signal == "매수" and self.balance >= current_price:
            possible_quantity = self.balance / current_price
            if possible_quantity > 0:
                trade_quantity = min(possible_quantity, 0.1)  # 최대 0.1 BTC로 제한
                cost = trade_quantity * current_price
                self.balance -= cost
                self.btc_held += trade_quantity
                return f"{trade_quantity:.8f} BTC 매수 @ {current_price:,.0f}원"
                
        elif signal == "매도" and self.btc_held > 0:
            trade_quantity = min(self.btc_held, 0.1)  # 최대 0.1 BTC로 제한
            revenue = trade_quantity * current_price
            self.balance += revenue
            self.btc_held -= trade_quantity
            return f"{trade_quantity:.8f} BTC 매도 @ {current_price:,.0f}원"
            
        return "관망"

class FeedbackLoop:
    def __init__(self):
        self.trade_history = []
        
    def log_trade(self, decision, price):
        """실제 거래 내역 기록"""
        if decision != "관망":
            trade_info = {
                "timestamp": datetime.datetime.now(),  # datetime.datetime.now() 사용
                "decision": decision,
                "price": price
            }
            self.trade_history.append(trade_info)
    
    def evaluate(self):
        """실제 수익 계산"""
        if not self.trade_history:
            return "거래 내역이 없습니다."
            
        total_profit = 0
        buy_amount = 0
        sell_amount = 0
        
        for trade in self.trade_history:
            price = trade["price"]
            if "매수" in trade["decision"]:
                quantity = float(trade["decision"].split(" ")[0])
                buy_amount += quantity * price
            elif "매도" in trade["decision"]:
                quantity = float(trade["decision"].split(" ")[0])
                sell_amount += quantity * price
        
        total_profit = sell_amount - buy_amount
        return f"총 수익: {total_profit:,.0f}원"

# 메인 실행 코드
if __name__ == "__main__":
    # 실제 비트코인 가격 데이터 조회
    price_data = get_bitcoin_price()
    if not price_data:
        print("비트코인 가격 데이터를 가져올 수 없습니다.")
        exit()
    
    # 시장 감성 분석
    sentiment = analyze_market_sentiment()
    
    # 매매 신호 생성
    signal = generate_signal(price_data, sentiment)
    
    # 거래 의사결정
    trader = TradingDecision(balance=10000000)  # 1천만원 초기자본
    decision = trader.make_decision(signal, price_data['current_price'])
    
    # 거래 실행
    executor = TradingExecutor()
    result = executor.execute_trade(decision)
    
    # 결과 기록 및 평가
    feedback = FeedbackLoop()
    feedback.log_trade(decision, price_data['current_price'])
    
    # 결과 출력
    print(f"현재가: {price_data['current_price']:,.0f}원")
    print(f"전일종가: {price_data['prev_closing_price']:,.0f}원")
    print(f"변화율: {price_data['change_rate']:.2f}%")
    print(f"감성점수: {sentiment:.2f}")
    print(f"매매신호: {signal}")
    print(f"실행결과: {result}")
    print(feedback.evaluate())

현재가: 122,250,000원
전일종가: 123,046,000원
변화율: 0.65%
감성점수: 0.19
매매신호: 관망
실행결과: 시뮬레이션: 관망 실행 완료
거래 내역이 없습니다.


In [44]:
import requests
import json
import time
from datetime import datetime
import re
import pyupbit
import pandas as pd
import numpy as np

class BitcoinTrader:
    def __init__(self, initial_balance=10000000):
        self.balance = initial_balance  # 초기 잔고 (원)
        self.btc_held = 0  # 보유 비트코인
        self.trade_history = []  # 거래 기록
        
    def get_bitcoin_price(self):
        """Upbit API를 통해 실제 비트코인 가격 조회"""
        try:
            current_price = pyupbit.get_current_price("KRW-BTC")
            if current_price:
                # 일봉 데이터 조회
                df = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=2)
                if df is not None and not df.empty:
                    prev_close = float(df['close'].iloc[-2])  # 전일 종가
                    today_open = float(df['open'].iloc[-1])   # 당일 시가
                    today_high = float(df['high'].iloc[-1])   # 당일 고가
                    today_low = float(df['low'].iloc[-1])     # 당일 저가
                    
                    return {
                        'current_price': float(current_price),
                        'prev_closing_price': prev_close,
                        'opening_price': today_open,
                        'high_price': today_high,
                        'low_price': today_low,
                        'change_rate': ((float(current_price) - prev_close) / prev_close) * 100
                    }
        except Exception as e:
            print(f"가격 조회 실패: {e}")
        return None

    def analyze_market(self, price_data):
        """시장 분석 및 감성 점수 계산"""
        if not price_data:
            return 0
        
        sentiment_score = 0
        
        # 가격 변화율 기반 점수 (30%)
        price_change = price_data['change_rate']
        sentiment_score += price_change * 0.3
        
        # 당일 시가 대비 현재가 상승여부 (20%)
        if price_data['current_price'] > price_data['opening_price']:
            sentiment_score += 0.2
        else:
            sentiment_score -= 0.2
            
        # 고가/저가 범위 내 현재가 위치 (30%)
        price_range = price_data['high_price'] - price_data['low_price']
        if price_range > 0:
            position = (price_data['current_price'] - price_data['low_price']) / price_range
            sentiment_score += (position - 0.5) * 0.3
            
        return max(min(sentiment_score, 1.0), -1.0)  # -1 ~ 1 사이로 정규화

    def generate_signal(self, price_data, sentiment_score):
        """매매 신호 생성"""
        if not price_data:
            return "관망"
        
        current_price = price_data['current_price']
        prev_closing = price_data['prev_closing_price']
        
        # 매수 신호 조건
        if (sentiment_score > 0.3 and 
            current_price < prev_closing * 0.97):  # 3% 이상 하락
            return "매수"
        
        # 매도 신호 조건
        elif (sentiment_score < -0.3 or 
              current_price > prev_closing * 1.05):  # 5% 이상 상승
            return "매도"
        
        return "관망"

    def execute_trade(self, signal, price_data):
        """매매 실행"""
        if not price_data or signal == "관망":
            return "관망"
            
        current_price = price_data['current_price']
        timestamp = datetime.now()
        
        if signal == "매수" and self.balance >= current_price:
            possible_quantity = self.balance / current_price
            if possible_quantity > 0:
                trade_quantity = min(possible_quantity, 0.1)  # 최대 0.1 BTC
                cost = trade_quantity * current_price
                self.balance -= cost
                self.btc_held += trade_quantity
                
                # 거래 기록
                self.trade_history.append({
                    "timestamp": timestamp,
                    "type": "매수",
                    "quantity": trade_quantity,
                    "price": current_price,
                    "total": cost
                })
                
                return f"{trade_quantity:.8f} BTC 매수 @ {current_price:,.0f}원"
                
        elif signal == "매도" and self.btc_held > 0:
            trade_quantity = min(self.btc_held, 0.1)  # 최대 0.1 BTC
            revenue = trade_quantity * current_price
            self.balance += revenue
            self.btc_held -= trade_quantity
            
            # 거래 기록
            self.trade_history.append({
                "timestamp": timestamp,
                "type": "매도",
                "quantity": trade_quantity,
                "price": current_price,
                "total": revenue
            })
            
            return f"{trade_quantity:.8f} BTC 매도 @ {current_price:,.0f}원"
            
        return "관망"

    def calculate_performance(self):
        """수익률 계산"""
        if not self.trade_history:
            return {
                "거래횟수": 0,
                "순수익": 0,
                "수익률": 0,
                "현재잔고": self.balance,
                "보유BTC": self.btc_held
            }
            
        total_buy = sum(trade["total"] for trade in self.trade_history if trade["type"] == "매수")
        total_sell = sum(trade["total"] for trade in self.trade_history if trade["type"] == "매도")
        net_profit = total_sell - total_buy
        
        # 현재 보유 중인 BTC의 평가금액 추가
        current_price = self.get_bitcoin_price()
        if current_price:
            btc_value = self.btc_held * current_price['current_price']
            total_value = self.balance + btc_value
            roi = ((total_value - 10000000) / 10000000) * 100  # 초기자본 대비 수익률
        else:
            roi = ((self.balance - 10000000) / 10000000) * 100
            
        return {
            "거래횟수": len(self.trade_history),
            "순수익": net_profit,
            "수익률": roi,
            "현재잔고": self.balance,
            "보유BTC": self.btc_held
        }

def main():
    # 트레이더 초기화
    trader = BitcoinTrader(initial_balance=10000000)
    
    print("비트코인 자동매매 프로그램을 시작합니다.")
    print("초기자본: 10,000,000원")
    print("Ctrl+C를 누르면 프로그램이 종료됩니다.")
    print("=" * 50)
    
    try:
        while True:
            # 가격 데이터 조회
            price_data = trader.get_bitcoin_price()
            if not price_data:
                print("가격 데이터를 가져올 수 없습니다.")
                time.sleep(1)
                continue
            
            # 시장 분석
            sentiment = trader.analyze_market(price_data)
            
            # 매매 신호 생성
            signal = trader.generate_signal(price_data, sentiment)
            
            # 매매 실행
            result = trader.execute_trade(signal, price_data)
            
            # 성과 분석
            performance = trader.calculate_performance()
            
            # 결과 출력
            print("\n" + "="*50)
            print(f"현재시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"현재가: {price_data['current_price']:,.0f}원")
            print(f"변화율: {price_data['change_rate']:.2f}%")
            print(f"감성점수: {sentiment:.2f}")
            print(f"매매신호: {signal}")
            print(f"실행결과: {result}")
            print(f"보유 BTC: {trader.btc_held:.8f}")
            print(f"보유 현금: {trader.balance:,.0f}원")
            print("\n[거래 성과]")
            for key, value in performance.items():
                if isinstance(value, float):
                    print(f"{key}: {value:,.2f}")
                else:
                    print(f"{key}: {value}")
            
            time.sleep(10)  # 10초 간격으로 업데이트
            
    except KeyboardInterrupt:
        print("\n프로그램을 종료합니다.")
        # 최종 성과 출력
        final_performance = trader.calculate_performance()
        print("\n[최종 거래 성과]")
        for key, value in final_performance.items():
            if isinstance(value, float):
                print(f"{key}: {value:,.2f}")
            else:
                print(f"{key}: {value}")
    
if __name__ == "__main__":
    main()

비트코인 자동매매 프로그램을 시작합니다.
초기자본: 10,000,000원
Ctrl+C를 누르면 프로그램이 종료됩니다.

현재시간: 2025-03-31 14:28:55
현재가: 122,406,000원
변화율: -0.52%
감성점수: -0.32
매매신호: 매도
실행결과: 관망
보유 BTC: 0.00000000
보유 현금: 10,000,000원

[거래 성과]
거래횟수: 0
순수익: 0
수익률: 0
현재잔고: 10000000
보유BTC: 0

현재시간: 2025-03-31 14:29:05
현재가: 122,405,000원
변화율: -0.52%
감성점수: -0.32
매매신호: 매도
실행결과: 관망
보유 BTC: 0.00000000
보유 현금: 10,000,000원

[거래 성과]
거래횟수: 0
순수익: 0
수익률: 0
현재잔고: 10000000
보유BTC: 0

현재시간: 2025-03-31 14:29:15
현재가: 122,398,000원
변화율: -0.53%
감성점수: -0.33
매매신호: 매도
실행결과: 관망
보유 BTC: 0.00000000
보유 현금: 10,000,000원

[거래 성과]
거래횟수: 0
순수익: 0
수익률: 0
현재잔고: 10000000
보유BTC: 0

현재시간: 2025-03-31 14:29:25
현재가: 122,397,000원
변화율: -0.53%
감성점수: -0.33
매매신호: 매도
실행결과: 관망
보유 BTC: 0.00000000
보유 현금: 10,000,000원

[거래 성과]
거래횟수: 0
순수익: 0
수익률: 0
현재잔고: 10000000
보유BTC: 0

현재시간: 2025-03-31 14:29:35
현재가: 122,400,000원
변화율: -0.53%
감성점수: -0.33
매매신호: 매도
실행결과: 관망
보유 BTC: 0.00000000
보유 현금: 10,000,000원

[거래 성과]
거래횟수: 0
순수익: 0
수익률: 0
현재잔고: 10000000
보유BTC: 0

현재시간: 2025-03-31 14:29:45
현재가: 12